---
title: C++ 网络编程 利用 socket 实现文件上传
tags: C++,Network|计算机网络,socket,upload,小书匠
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ 网络编程 利用 socket 实现文件上传

这个代码实现了十分简单的文件上传功能。主要是练习简单的 socket 的使用。

## 代码

### server.cpp

In [5]:
%%file server.cpp
#include <iostream>
#include <sys/socket.h>
#include <cstring>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <fstream>
#include <unistd.h>

#define IP "127.0.0.1"
#define PORT 5000
#define BUFF_SIZE 1000

using namespace std;

int main()
{
    int server_sock = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP); // 定义 socket

    sockaddr_in server_addr; // 定义 ip
    memset(&server_addr, 0, sizeof(server_addr));
    server_addr.sin_family = AF_INET;
    server_addr.sin_addr.s_addr = inet_addr(IP);
    server_addr.sin_port = htons(PORT);

    bind(server_sock, (sockaddr*)&server_addr, sizeof(server_addr)); // 绑定地址
    
    listen(server_sock, 20);
    
    sockaddr_in client_addr;
    socklen_t client_addr_size = sizeof(client_addr);
    
    int client_sock = accept(server_sock, (sockaddr*)&client_addr, &client_addr_size);
    
    char buff[BUFF_SIZE];
    recv(client_sock, buff, BUFF_SIZE, 0); // 得到
    string filename = buff;
    ofstream fout(filename, ios_base::binary);
    if (!fout)
    {
        cout << "Can't open file " << filename << endl;
        close(server_sock);
        close(client_sock);
        return 0;
    }
    
    int n_recv_total = 0;
    int n_recv = 0;
    while (1)
    {
        n_recv = recv(client_sock, buff, BUFF_SIZE, 0);
        if (n_recv == 0)
        {
            break;
        }
        fout.write(buff, n_recv);
        n_recv_total += n_recv;
        cout << "[Server] " << n_recv << " bytes has been recieved from client\n";
    }
    fout.close();
    cout << "file " << filename << " saved successfully!\n";
    cout << "Finish save file " << filename << " total_size: " << n_recv_total << " bytes speed: " << float(n_recv_total) / 1024 << " kbs" << endl;
    close(server_sock);
    close(client_sock);
    return 0;
}

Overwriting server.cpp


### client.cpp

In [6]:
%%file client.cpp
#include <iostream>
#include <sys/socket.h>
#include <cstring>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <fstream>
#include <unistd.h>

#define IP "127.0.0.1"
#define PORT 5000
#define BUFF_SIZE 1000

int main()
{
    int server_sock = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP); // 定义 socket

    sockaddr_in server_addr; // 定义 ip
    memset(&server_addr, 0, sizeof(server_addr));
    server_addr.sin_family = AF_INET;
    server_addr.sin_addr.s_addr = inet_addr(IP);
    server_addr.sin_port = htons(PORT);

    connect(server_sock, (sockaddr*)&server_addr, sizeof(server_addr)); // 连接服务端

    std::cout << "Input the path of file you want to upload:" << std::endl;
    std::string filename;
    getline(std::cin, filename);

    std::string save_filename;
    std::cout << "Input the filename you want to save:" << std::endl;
    getline(std::cin, save_filename);

    send(server_sock, save_filename.c_str(), save_filename.size()+1, 0); // 发送给服务端保存的文件名

    char buff[BUFF_SIZE];
    std::ifstream fin(filename, std::ios_base::binary);
    if (!fin) 
    {
        std::cout << "Can't open file " << filename << std::endl;
        fin.close();
        close(server_sock);
        return 0;
    }
    int n_send_total = 0;
    int n_send = 0;
    while (fin)
    {
        fin.read(buff, sizeof(buff));
        n_send = send(server_sock, buff, sizeof(buff), 0);
        if (n_send <= 0) break;
        n_send_total += n_send;
        std::cout << "[Client] send to server " << n_send << " bytes" <<  std::endl;
        memset(buff, 0, sizeof(buff));
    }
    std::cout << "Finish upload file " << filename << " total_size: " << n_send_total << " bytes speed: " << float(n_send_total) / 1024 << " kbs" << std::endl;
    fin.close();
    close(server_sock);
    return 0;
}

Overwriting client.cpp


### 编译

In [7]:
!g++ server.cpp -o server

In [8]:
!g++ client.cpp -o client

# References

[socket文件传输功能的实现_C语言中文网](http://c.biancheng.net/cpp/html/3045.html)